# Learning how to build deep neural nets
## Based on Chap 10 of Hands on ML 3rd ed

In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
np.random.seed(0)
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# There's some numpy incompatibility with tf which will probably get fixed soon. Downgrading to 1.16.1 is a temporary fix
np.__version__

'1.18.1'

### List of initializer functions, glorot_uniform is usually the default.
### He initializer is also one that works well with Relu

In [3]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'Identity',
 'Initializer',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'serialize',
 'zeros']

In [5]:
keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

In [6]:
# To get He initializer with uniform distribution but based on fan_avg rather than fan_in
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

### List of activation functions

In [7]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'tanh']

1. Sigmoid function is in biological neurons but lead to exploding and vanishing gradients so sometimes ReLU is better.
2. Drawback of ReLU - dying neurons, stop outputting anything other than zero
3. To solve this - leaky ReLU = max(\alpha z , z) alpha usually set to 0.01
4. Newer paper used something called exponential linear unit (ELU) - slower to compute
5. 2017 - Scaled ELU (SELU). Exclusively a stack of dense layers which self normalize. Initialization with lecun_normal. Architecture must be sequential, can improve CNN's as well

In [8]:
# Can't set as activation directly 
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

### Try to train fashion MNIST using Leaky ReLU

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [11]:
tf.random.set_seed(0)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    # Have to use this additional layer after each layer
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [12]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])


In [13]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 5s 100us/sample - loss: 1.2530 - accuracy: 0.6110 - val_loss: 0.8560 - val_accuracy: 0.7272
Epoch 2/10
55000/55000 [==============================] - 5s 86us/sample - loss: 0.7813 - accuracy: 0.7411 - val_loss: 0.6964 - val_accuracy: 0.7694
Epoch 3/10
55000/55000 [==============================] - 5s 86us/sample - loss: 0.6735 - accuracy: 0.7742 - val_loss: 0.6251 - val_accuracy: 0.7982
Epoch 4/10
55000/55000 [==============================] - 5s 84us/sample - loss: 0.6162 - accuracy: 0.7950 - val_loss: 0.5851 - val_accuracy: 0.8056
Epoch 5/10
55000/55000 [==============================] - 5s 83us/sample - loss: 0.5788 - accuracy: 0.8086 - val_loss: 0.5528 - val_accuracy: 0.8180
Epoch 6/10
55000/55000 [==============================] - 5s 82us/sample - loss: 0.5523 - accuracy: 0.8160 - val_loss: 0.5279 - val_accuracy: 0.8272
Epoch 7/10
55000/55000 [==============================] 

### Training using SELU
In this case need to scale to mean 0 and SD 1

In [14]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [17]:
# Much slower but avoids vanishing or exploding gradient problem
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 53s 961us/sample - loss: 1.2577 - accuracy: 0.5144 - val_loss: 0.9184 - val_accuracy: 0.6616
Epoch 2/5
55000/55000 [==============================] - 45s 825us/sample - loss: 0.8798 - accuracy: 0.6705 - val_loss: 1.6595 - val_accuracy: 0.2790
Epoch 3/5
55000/55000 [==============================] - 45s 823us/sample - loss: 0.9829 - accuracy: 0.6283 - val_loss: 0.7660 - val_accuracy: 0.7002
Epoch 4/5
55000/55000 [==============================] - 46s 836us/sample - loss: 0.7512 - accuracy: 0.7079 - val_loss: 0.6577 - val_accuracy: 0.7442
Epoch 5/5
55000/55000 [==============================] - 45s 816us/sample - loss: 0.6259 - accuracy: 0.7650 - val_loss: 0.5664 - val_accuracy: 0.7994


### Batch normalization
1. Adding an operation just before activation to zero center and normalize each input. 
2. Can add at the start too, then won't require scaling variables (normalizes each batch btw not the whole dataset)
3. Instead of doing normalization each time, keras uses a moving average of layers means and SDs
4. Using batch normalization helps with vanishing gradients problem, so can use tanh and sigmoid activation too
5. Also acts as a regularizer, reducing need for other regularizers like dropout
6. Faster convergence

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [5]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 11s 207us/sample - loss: 0.8603 - accuracy: 0.7109 - val_loss: 0.5515 - val_accuracy: 0.8180
Epoch 2/10
55000/55000 [==============================] - 10s 191us/sample - loss: 0.5737 - accuracy: 0.8017 - val_loss: 0.4728 - val_accuracy: 0.8400
Epoch 3/10
55000/55000 [==============================] - 10s 184us/sample - loss: 0.5144 - accuracy: 0.8215 - val_loss: 0.4359 - val_accuracy: 0.8540
Epoch 4/10
55000/55000 [==============================] - 10s 185us/sample - loss: 0.4791 - accuracy: 0.8335 - val_loss: 0.4153 - val_accuracy: 0.8588
Epoch 5/10
55000/55000 [==============================] - 12s 211us/sample - loss: 0.4580 - accuracy: 0.8388 - val_loss: 0.4013 - val_accuracy: 0.8642
Epoch 6/10
55000/55000 [==============================] - 11s 196us/sample - loss: 0.4363 - accuracy: 0.8480 - val_loss: 0.3904 - val_accuracy: 0.8640
Epoch 7/10
55000/55000 [=====================

Layer before a batch normalization doesn't need bias terms (waste of space since BN does it automatically) 
So can do something like this

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [9]:

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 18s 332us/sample - loss: 1.0344 - accuracy: 0.6736 - val_loss: 0.6724 - val_accuracy: 0.7922
Epoch 2/10
55000/55000 [==============================] - 16s 287us/sample - loss: 0.6782 - accuracy: 0.7827 - val_loss: 0.5544 - val_accuracy: 0.8228
Epoch 3/10
55000/55000 [==============================] - 15s 281us/sample - loss: 0.5910 - accuracy: 0.8052 - val_loss: 0.4973 - val_accuracy: 0.8388
Epoch 4/10
55000/55000 [==============================] - 15s 277us/sample - loss: 0.5440 - accuracy: 0.8173 - val_loss: 0.4638 - val_accuracy: 0.8458
Epoch 5/10
55000/55000 [==============================] - 15s 267us/sample - loss: 0.5093 - accuracy: 0.8277 - val_loss: 0.4418 - val_accuracy: 0.8532
Epoch 6/10
55000/55000 [==============================] - 14s 258us/sample - loss: 0.4899 - accuracy: 0.8325 - val_loss: 0.4234 - val_accuracy: 0.8582
Epoch 7/10
55000/55000 [=====================

# Transfer learning

### Reusing pretrained layers
1. Not a good idea to train very large DNN from scratch
2. Try to find existing NN that accomplishes a similar task and reuse the lower layers
3. Speed up training and need less training data

### Example
1. Use MNIST Fashion with 8 classes except sandal and shirt - someone built a model on it and got 90% accuracy
2. Now use the sandal and shirt data, train binary classifier, small dataset only 200 images, 
3. Task quite similar so transfer learning between the first model to this

In [10]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [11]:
# except sandals and shirt
X_train_A.shape

(43986, 28, 28)

In [13]:
# only sandals and shirt
X_train_B.shape

(200, 28, 28)

In [14]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [15]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [16]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [17]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [18]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Train on 43986 samples, validate on 4014 samples
Epoch 1/20
43986/43986 [==============================] - 10s 232us/sample - loss: 0.5712 - accuracy: 0.8140 - val_loss: 0.3755 - val_accuracy: 0.8747
Epoch 2/20
43986/43986 [==============================] - 9s 202us/sample - loss: 0.3577 - accuracy: 0.8763 - val_loss: 0.3247 - val_accuracy: 0.8901
Epoch 3/20
43986/43986 [==============================] - 8s 183us/sample - loss: 0.3218 - accuracy: 0.8885 - val_loss: 0.3009 - val_accuracy: 0.8989
Epoch 4/20
43986/43986 [==============================] - 8s 182us/sample - loss: 0.3021 - accuracy: 0.8951 - val_loss: 0.2868 - val_accuracy: 0.9028
Epoch 5/20
43986/43986 [==============================] - 9s 200us/sample - loss: 0.2887 - accuracy: 0.9006 - val_loss: 0.2785 - val_accuracy: 0.9088
Epoch 6/20
43986/43986 [==============================] - 9s 200us/sample - loss: 0.2781 - accuracy: 0.9045 - val_loss: 0.2714 - val_accuracy: 0.9098
Epoch 7/20
43986/43986 [==========================

In [19]:
model_A.save("../models/model_A_TL.h5")

### Try a model with the small training set only 

In [20]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [21]:

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [22]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/20
200/200 [==============================] - 2s 9ms/sample - loss: 0.4965 - accuracy: 0.7850 - val_loss: 0.3920 - val_accuracy: 0.8590
Epoch 2/20
200/200 [==============================] - 0s 945us/sample - loss: 0.3490 - accuracy: 0.8650 - val_loss: 0.2931 - val_accuracy: 0.9087
Epoch 3/20
200/200 [==============================] - 0s 932us/sample - loss: 0.2597 - accuracy: 0.9250 - val_loss: 0.2352 - val_accuracy: 0.9422
Epoch 4/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.2058 - accuracy: 0.9650 - val_loss: 0.1982 - val_accuracy: 0.9615
Epoch 5/20
200/200 [==============================] - 0s 858us/sample - loss: 0.1710 - accuracy: 0.9750 - val_loss: 0.1731 - val_accuracy: 0.9645
Epoch 6/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.1462 - accuracy: 0.9850 - val_loss: 0.1528 - val_accuracy: 0.9696
Epoch 7/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.1271 -

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 784)               3136      
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235200    
_________________________________________________________________
batch_normalization_4 (Batch (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30000     
_________________________________________________________________
batch_normalization_5 (Batch (None, 100)              

### What if we use transfer learning

In [26]:
model_A = keras.models.load_model("../models/model_A_TL.h5")
# Use all layers except the last (different shape)
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# Add in a final layer
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [27]:
# model A and B share layers, changes to model B on A will affect model A too, clone the model to avoid this
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [28]:
# Don't want to train already trained layers from model_A
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [29]:
# Since the new output layer was initialized randomly, will make large errors, so freeze the reused layers during first few epochs 
# gives the new layer time to learn some reasonable weights
# always compile after freeze or unfreeze layers
# After unfreezing, good idea to reduce the learning rate
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-4),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 1s 7ms/sample - loss: 0.0805 - accuracy: 0.9850 - val_loss: 0.0918 - val_accuracy: 0.9868
Epoch 2/4
200/200 [==============================] - 0s 723us/sample - loss: 0.0795 - accuracy: 0.9850 - val_loss: 0.0909 - val_accuracy: 0.9868
Epoch 3/4
200/200 [==============================] - 0s 768us/sample - loss: 0.0786 - accuracy: 0.9850 - val_loss: 0.0899 - val_accuracy: 0.9868
Epoch 4/4
200/200 [==============================] - 0s 952us/sample - loss: 0.0776 - accuracy: 0.9850 - val_loss: 0.0890 - val_accuracy: 0.9868
Train on 200 samples, validate on 986 samples
Epoch 1/16
200/200 [==============================] - 2s 9ms/sample - loss: 0.0768 - accuracy: 0.9850 - val_loss: 0.0884 - val_accuracy: 0.9868
Epoch 2/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.0761 - accuracy: 0.9850 - val_loss: 0.0877 - val_accuracy: 0.9868
Epoch 3/16
200/200 [======================

In [30]:
model_B.evaluate(X_test_B, y_test_B)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.06474531203508377, 0.9885]

In [31]:
model_B_on_A.evaluate(X_test_B, y_test_B)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.08566666263341903, 0.9855]